In [114]:
%%html
<style type='text/css'>
.CodeMirror{
font-size: 12pt;
font-family: Fira Code Light
}
</style>

In [115]:
import black
import jupyter_black

jupyter_black.load(
    lab=True,
    line_length=79,
    verbosity="DEBUG",
    target_version=black.TargetVersion.PY38,
)

In [116]:
%matplotlib notebook
import sympy.stats
import sympy as sym
import scipy as sp
import numpy as np
import matplotlib.pyplot as plt
from sympy.utilities.lambdify import lambdastr
# from mayavi import mlab
# from tvtk.api import tvtk

In [117]:
from IPython.display import Math


def show(*exprs):
    s = r"\large\begin{array}{ll}"
    for e in exprs:
        if not isinstance(e, str):
            e = sym.interactive.printing.default_latex(e)
        s += e + r" &\\"
    s += r"\end{array}"
    return Math(s)

# Solution for mid +/- offset

In [118]:
ma, m, f = sym.var(
    r"\bar{a} m f",
    real=True,
)

md, sa, sd, f, la = sym.var(
    r"\bar{\delta} \langle{a}\rangle \langle{\delta}\rangle f \lambda",
    positive=True,
)

mx, my, mz = sym.var(
    r"\bar{\xi} \bar{\psi} \bar{\zeta}",
    real=True,
)

sx, sy, sz = sym.var(
    r"\langle{\xi}\rangle \langle{\psi}\rangle " r"\langle{\zeta}\rangle",
    positive=True,
)

xi, psi, zeta = sym.var(
    r"x y z"
)

var_xi, var_psi, var_zeta = sym.var(
    r"vx vy vz"
)

In [119]:
x = my - mx
y = my
z = my + mz

tsx = sy + sx
tsy = sy
tsz = sy + sz

In [120]:
ex = ma + md * (m - f/2)
ey = ma + md * (m)
ez = ma + md * (m + f/2)

esx = sa + sd * (m - f/2) ** 2
esy = sa + sd * (m) ** 2
esz = sa + sd * (m + f/2) ** 2

In [121]:
expr = (
    la*(
    (ex - x) ** 2
    + (ey - y) ** 2
    + (ez - z) ** 2
    )
    + (esx - tsx) ** 2
    + (esy - tsy) ** 2
    + (esz - tsz) ** 2
)

In [122]:
dma = expr.diff(ma).simplify()
dmd = expr.diff(md).simplify()
dsa = expr.diff(sa).simplify()
dsd = expr.diff(sd).simplify()
dm = expr.diff(m).simplify()
df = expr.diff(f).simplify()
dla = expr.diff(la).simplify()

In [123]:
# lambda is an parameter, not a variable
dla

(\bar{\delta}*m - \bar{\psi} + \bar{a})**2 + (\bar{\delta}*(f - 2*m) + 2*\bar{\psi} - 2*\bar{\xi} - 2*\bar{a})**2/4 + (\bar{\delta}*(f + 2*m) - 2*\bar{\psi} - 2*\bar{\zeta} + 2*\bar{a})**2/4

In [124]:
%%time
# solving without lambda in variable list, and in differentiation list
solutions = sym.solve([dma, dmd, dsa, dsd, dm, df], [ma, md, sa, sd, m, f], dict=True)
len(solutions)

CPU times: user 6.44 s, sys: 0 ns, total: 6.44 s
Wall time: 6.44 s


1

In [125]:
# f ~ m, so choose f=1
solutions[0][f]

4*m*(-\langle{\xi}\rangle - \langle{\zeta}\rangle)/(\langle{\xi}\rangle - \langle{\zeta}\rangle)

## Case f=1

In [126]:
offset = sym.Rational(1, 2)

In [127]:
ex = ma + md * (m - offset)
ey = ma + md * (m)
ez = ma + md * (m + offset)

esx = sa + sd * (m - offset) ** 2
esy = sa + sd * (m) ** 2
esz = sa + sd * (m + offset) ** 2

In [128]:
expr = (
    la*(
    (ex - x) ** 2
    + (ey - y) ** 2
    + (ez - z) ** 2
    )
    + (esx - tsx) ** 2
    + (esy - tsy) ** 2
    + (esz - tsz) ** 2
)

In [129]:
dma = expr.diff(ma).simplify()
dmd = expr.diff(md).simplify()
dsa = expr.diff(sa).simplify()
dsd = expr.diff(sd).simplify()
dm = expr.diff(m).simplify()

In [130]:
%%time
# solving without `f` in variable list, and in differentiation list
solutions = sym.solve([dma, dmd, dsa, dsd, dm], [ma, md, sa, sd, m], dict=True)
len(solutions)

CPU times: user 375 ms, sys: 0 ns, total: 375 ms
Wall time: 381 ms


1

In [131]:
# no lambda in formulas
show(*[
    sym.Eq(vr, solutions[0][vr])
    for vr in [ma, md, sa, sd, m]
])

<IPython.core.display.Math object>

## Case f=1, lambda=1

In [132]:
offset = sym.Rational(1, 2)
ex = ma + md * (m - offset)
ey = ma + md * (m)
ez = ma + md * (m + offset)

esx = sa + sd * (m - offset) ** 2
esy = sa + sd * (m) ** 2
esz = sa + sd * (m + offset) ** 2

In [133]:
expr = (
    1*(
    (ex - x) ** 2
    + (ey - y) ** 2
    + (ez - z) ** 2
    )
    + (esx - tsx) ** 2
    + (esy - tsy) ** 2
    + (esz - tsz) ** 2
)

In [134]:
dma = expr.diff(ma).simplify()
dmd = expr.diff(md).simplify()
dsa = expr.diff(sa).simplify()
dsd = expr.diff(sd).simplify()
dm = expr.diff(m).simplify()

In [135]:
%%time
# solving without `f` in variable list, and in differentiation list
solutions = sym.solve([dma, dmd, dsa, dsd, dm], [ma, md, sa, sd, m], dict=True)
len(solutions)

CPU times: user 288 ms, sys: 0 ns, total: 288 ms
Wall time: 292 ms


1

In [136]:
# solution are done
show(*[
 sym.Eq(vr, solutions[0][vr])
    for vr in [ma, md, sa, sd, m]
])

<IPython.core.display.Math object>

In [137]:
subs_list = [(mx, xi), (my, psi), (mz, zeta), (sx, var_xi), (sy, var_psi), (sz, var_zeta)]

lambdastr((xi, psi, zeta, var_xi, var_psi, var_zeta), [
    solutions[0][vr].subs(subs_list)
    for vr in [ma, md, sa, sd, m]
])

'lambda x,y,z,vx,vy,vz: ([(1/12)*(-vx*x + 12*vx*y + 7*vx*z - 7*vz*x + 12*vz*y + vz*z)/(vx + vz), x + z, (1/8)*(-vx**2 + 8*vx*vy + 2*vx*vz + 8*vy*vz - vz**2)/(vx + vz), 2*vx + 2*vz, (-vx + vz)/(4*vx + 4*vz)])'

# Solution for left, mid, right

In [138]:
ex = ma + md * (m - f/2)
ey = ma + md * (m)
ez = ma + md * (m + f/2)

esx = sa + sd * (m - f/2) ** 2
esy = sa + sd * (m) ** 2
esz = sa + sd * (m + f/2) ** 2

In [139]:
expr = (
    la*(
    (ex - mx) ** 2
    + (ey - my) ** 2
    + (ez - mz) ** 2
    )
    + (esx - sx) ** 2
    + (esy - sy) ** 2
    + (esz - sz) ** 2
)

In [140]:
dma = expr.diff(ma).simplify()
dmd = expr.diff(md).simplify()
dsa = expr.diff(sa).simplify()
dsd = expr.diff(sd).simplify()
dm = expr.diff(m).simplify()
df = expr.diff(f).simplify()

In [141]:
%%time
solutions = sym.solve([dma, dmd, dsa, dsd, dm, df], [ma, md, sa, sd, m, f], dict=True)
len(solutions)

CPU times: user 51.1 s, sys: 7.77 ms, total: 51.1 s
Wall time: 51.2 s


1

In [142]:
# f ~ m, so choose f=1
# no lambda in formulas, so lambda = 1
show(*[
 sym.Eq(vr, solutions[0][vr])
    for vr in [ma, md, sa, sd, f]
])

<IPython.core.display.Math object>

## Case f=1 lambda=1

In [149]:
offset = sym.Rational(1, 2)
ex = ma + md * (m - offset)
ey = ma + md * (m)
ez = ma + md * (m + offset)

esx = sa + sd * (m - offset) ** 2
esy = sa + sd * (m) ** 2
esz = sa + sd * (m + offset) ** 2

In [150]:
expr = (
    1*(
    (ex - mx) ** 2
    + (ey - my) ** 2
    + (ez - mz) ** 2
    )
    + (esx - sx) ** 2
    + (esy - sy) ** 2
    + (esz - sz) ** 2
)

In [151]:
dma = expr.diff(ma).simplify()
dmd = expr.diff(md).simplify()
dsa = expr.diff(sa).simplify()
dsd = expr.diff(sd).simplify()
dm = expr.diff(m).simplify()

In [152]:
%%time
# no `f` variable
solutions = sym.solve([dma, dmd, dsa, dsd, dm], [ma, md, sa, sd, m], dict=True)
len(solutions)

CPU times: user 352 ms, sys: 0 ns, total: 352 ms
Wall time: 355 ms


1

In [155]:
show(*[
 sym.Eq(vr, solutions[0][vr])
    for vr in [ma, md, sa, sd, m]
])

<IPython.core.display.Math object>

In [148]:
subs_list = [(mx, xi), (my, psi), (mz, zeta), (sx, var_xi), (sy, var_psi), (sz, var_zeta)]

lambdastr((xi, psi, zeta, var_xi, var_psi, var_zeta), [
    solutions[0][vr].subs(subs_list)
    for vr in [ma, md, sa, sd, m]
])

'lambda x,y,z,vx,vy,vz: ([(1/12)*(-vx*x - 4*vx*y - 7*vx*z + 8*vy*x + 8*vy*y + 8*vy*z - 7*vz*x - 4*vz*y - vz*z)/(-vx + 2*vy - vz), -x + z, (1/8)*(vx**2 - 8*vx*vy - 2*vx*vz + 16*vy**2 - 8*vy*vz + vz**2)/(-vx + 2*vy - vz), 2*vx - 4*vy + 2*vz, (vx - vz)/(-4*vx + 8*vy - 4*vz)])'